<a href="https://colab.research.google.com/github/ricardoskewes/68610finalprojectgroup48/blob/colab_branch/thoughtwasfinalbutnocode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
rm -rf 68610finalprojectgroup48
git clone https://github.com/ricardoskewes/68610finalprojectgroup48.git

Cloning into '68610finalprojectgroup48'...


In [ ]:
!pip install sacrebleu simpletransformers
!pip install wordfreq
!pip install wandb


In [ ]:

# Append the cloned repository to sys.path

import sys
sys.path.append("/content/68610finalprojectgroup48")
projectDir = "/content/68610finalprojectgroup48/tweepfake"
sys.path.insert(0, projectDir)
resultsDir = projectDir+"/data/results"
import datetime
import math
import numpy as np
import pandas as pd
import torch
import wandb
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob
from wordfreq import word_frequency
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from transformers import (BertForSequenceClassification, BertTokenizerFast, Trainer,
                          TrainingArguments, EarlyStoppingCallback, DataCollatorWithPadding)
from datasets import Dataset
import random
# from simpletransformers.classification import ClassificationModel, ClassificationArgs

# wandb library
import wandb

#DataHandler
from DataHandler import DataHandler

from datasets import Dataset




In [ ]:
# Paths to datasets
csvTrainDataset = f"{projectDir}/data/splits/train.csv"
csvValDataset = f"{projectDir}/data/splits/validation.csv"
csvTestDataset = f"{projectDir}/data/splits/test.csv"

# Load datasets
dfTrain = dh.readCSVData(csvTrainDataset)
dfVal = dh.readCSVData(csvValDataset)
dfTest = dh.readCSVData(csvTestDataset)

# Select relevant columns
dfTrainDataset = dfTrain[["screen_name", "text", "account.type"]]
dfValDataset = dfVal[["screen_name", "text", "account.type"]]
dfTestDataset = dfTest[["screen_name", "text", "account.type"]]

# Prepare training data
X_train_all = dfTrainDataset.drop(columns=['screen_name'])
X_train_all.columns = ["text", "label"]

X_val_all = dfValDataset.drop(columns=['screen_name'])
X_val_all.columns = ["text", "label"]

X_test_all = dfTestDataset.drop(columns=['screen_name'])
X_test_all.columns = ["text", "label"]

# Map labels to integers
dictLabels = {"human": 0, "bot": 1}
X_train_all["label"] = X_train_all["label"].map(dictLabels)
X_val_all["label"] = X_val_all["label"].map(dictLabels)
X_test_all["label"] = X_test_all["label"].map(dictLabels)

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(X_train_all, preserve_index=False)
val_dataset = Dataset.from_pandas(X_val_all, preserve_index=False)
test_dataset = Dataset.from_pandas(X_test_all, preserve_index=False)

# Verify the conversion
print(type(train_dataset))  # Should output: <class 'datasets.arrow_dataset.Dataset'>
print(type(val_dataset))    # Should output: <class 'datasets.arrow_dataset.Dataset'>
print(type(test_dataset))   # Should output: <class 'datasets.arrow_dataset.Dataset'>

# -----------------------
# Feature Functions
# -----------------------

def get_spacy_doc(text):
    return spacy_nlp(text)

def switch_spacy_to_text(func):
    def inner1(*args, **kwargs):
        spacy_doc = get_spacy_doc(args[0])
        return func(spacy_doc, **kwargs)
    return inner1

def count_words(text):
    return len(text.split())

def avg_word_length(text):
    words = text.split()
    return np.mean([len(w) for w in words]) if len(words) > 0 else 0

@switch_spacy_to_text
def get_ANP(spacy_doc):
    filtered_tokens = [token for token in spacy_doc if not token.is_punct and not token.is_space]
    total_tokens = len(filtered_tokens)

    adjectives = [token.text for token in filtered_tokens if token.pos_ == "ADJ"]
    nouns = [token.text for token in filtered_tokens if token.pos_ == "NOUN"]
    pronouns = [token.text for token in filtered_tokens if token.pos_ == "PRON"]

    if total_tokens == 0:
        adj_density = 0
        noun_density = 0
        pronoun_density = 0
    else:
        adj_density = len(adjectives) / total_tokens
        noun_density = len(nouns) / total_tokens
        pronoun_density = len(pronouns) / total_tokens
    return adjectives, nouns, pronouns, adj_density, noun_density, pronoun_density

def get_ANP_clean(text):
    _, _, _, adj_density, noun_density, pronoun_density = get_ANP(text)
    return adj_density, noun_density, pronoun_density

@switch_spacy_to_text
def get_capitalizations(spacy_doc):
    capitalizations = [token.text for token in spacy_doc if token.text.isupper()]
    return capitalizations, len(capitalizations) / len(spacy_doc) if len(spacy_doc) > 0 else 0

def get_capitalizations_clean(text):
    _, cap_ratio = get_capitalizations(text)
    return np.round(cap_ratio, 2)

@switch_spacy_to_text
def get_sentiment(spacy_doc):
    polarity, subjectivity = TextBlob(spacy_doc.text).sentiment
    return np.round(polarity, 3), np.round(subjectivity, 3)

@switch_spacy_to_text
def calculate_rarity_scores(spacy_doc, lang='en'):
    adj_rarity_scores = []
    noun_rarity_scores = []
    for token in spacy_doc:
        if token.pos_ == "ADJ":
            freq = word_frequency(token.text.lower(), lang)
            adj_rarity_scores.append(-math.log(freq) if freq > 0 else 0)
        if token.pos_ == "NOUN":
            freq = word_frequency(token.text.lower(), lang)
            noun_rarity_scores.append(-math.log(freq) if freq > 0 else 0)
    res = [0, 0]
    if noun_rarity_scores:
        res[0] = np.median(noun_rarity_scores)
    if adj_rarity_scores:
        res[1] = np.median(adj_rarity_scores)
    return tuple(np.round(res, 3))

@switch_spacy_to_text
def get_punctuation(spacy_doc):
    punctuation = [token.text for token in spacy_doc if token.pos_ == "PUNCT"]
    return punctuation, len(punctuation)/len(spacy_doc) if len(spacy_doc)>0 else 0

def get_punctuation_clean(text):
    _, punct_ratio = get_punctuation(text)
    return np.round(punct_ratio, 2)

# Dictionary of classical features
C = {
    1: count_words,                  # C1: Number of words in sentence
    2: avg_word_length,              # C2: Average word length in sentence
    3: get_ANP_clean,                # C3: Density scores of adjectives, pronouns, and nouns
    4: get_capitalizations_clean,     # C4: Density of capital letters
    5: get_sentiment,                # C5: Sentiment analysis
    6: calculate_rarity_scores,      # C6: Noun and adjective rarity scores
    7: get_punctuation_clean         # C7: Density of punctuation
}

# Model descriptions
model_descriptions = {
    1: 'BERT*(tweet)',
    2: 'BERT*({tweet <SEP> C1 <SEP> C2 ...})',
    3: 'BERT*({tweet <FEAT> C1 <FEAT> C2 ...})',
}

# Model-specific arguments
model_args_dict = {
    1: {'transformer_type': 'bert', 'transformer_name': 'bert-base-cased', 'output_dir': 'type1'},
    2: {'transformer_type': 'bert', 'transformer_name': 'bert-base-cased', 'output_dir': 'type2'},
    3: {'separator_token_name': 'FEAT', 'transformer_type': 'bert', 'transformer_name': 'bert-base-cased', 'output_dir': 'type3'},
}

# Hyperparameters for each model
model_hyperparameters = {
    1: {"learning_rate": 1e-5, "train_batch_size": 16, "num_train_epochs": 3},
    2: {"learning_rate": 1e-5, "train_batch_size": 16, "num_train_epochs": 3, "C_ids": [1,2]},
    3: {"learning_rate": 1e-5, "train_batch_size": 16, "num_train_epochs": 3, "C_ids": [3,4,5]},
}

# -----------------------
# Special Tokens and Encoding Functions
# -----------------------

def create_special_tokens_dict(model_id, model_args_dict):
    additional_tokens = []
    if model_id == 3:
        additional_tokens.append(model_args_dict[model_id]['separator_token_name'])
    return {'additional_special_tokens': additional_tokens}

def initialize_model_with_special_tokens(model_id, model_args_dict, hyperparameters, special_tokens_dict, model_descriptions):
    transformer_name = model_args_dict[model_id]['transformer_name']
    tokenizer = BertTokenizerFast.from_pretrained(transformer_name)
    model = BertForSequenceClassification.from_pretrained(transformer_name, num_labels=2)

    if special_tokens_dict and len(special_tokens_dict['additional_special_tokens']) > 0:
        tokenizer.add_special_tokens(special_tokens_dict)
        model.resize_token_embeddings(len(tokenizer))

    return model, tokenizer

def encode_with_tokens_based_on_model_id(example, tokenizer, model_id, model_args_dict, model_hyperparameters):
    tweet = example['text']
    if 'separator_token_name' in model_args_dict[model_id]:
        feat_sep_token = model_args_dict[model_id]['separator_token_name']
        feat_sep_token_id = tokenizer.convert_tokens_to_ids(feat_sep_token)
    else:
        feat_sep_token_id = tokenizer.sep_token_id or tokenizer.eos_token_id

    if 'C_ids' in model_hyperparameters[model_id]:
        C_functions = [C[i] for i in model_hyperparameters[model_id]['C_ids']]
    elif 'C_ids' in model_args_dict[model_id]:
        C_functions = [C[i] for i in model_args_dict[model_id]['C_ids']]
    else:
        C_functions = None

    # Encode features
    feature_tokens = []
    if C_functions is not None:
        for func in C_functions:
            feature_tokens.append(feat_sep_token_id)
            feature_value = func(tweet)
            if isinstance(feature_value, tuple):
                feature_value = ' '.join(map(str, feature_value))
            else:
                feature_value = str(feature_value)
            # Tokenize feature value
            encoded_feature = tokenizer.encode(
                feature_value,
                add_special_tokens=False,
                truncation=True,
                max_length=10  # Adjust as needed
            )
            feature_tokens.extend(encoded_feature)

    # Concatenate tweet and features with separator tokens
    if model_id == 2:
        # Model 2: BERT*({tweet <SEP> C1 <SEP> C2 ...})
        encoded_tweet = tokenizer.encode(
            tweet,
            add_special_tokens=True,
            truncation=True,
            max_length=tokenizer.model_max_length - len(feature_tokens) - 2  # Reserve for two <SEP>
        )
        encoded_sequence = encoded_tweet + [feat_sep_token_id] + feature_tokens
    elif model_id == 3:
        # Model 3: BERT*({tweet <FEAT> C1 <FEAT> C2 ...})
        encoded_tweet = tokenizer.encode(
            tweet,
            add_special_tokens=True,
            truncation=True,
            max_length=tokenizer.model_max_length - len(feature_tokens) - 2  # Reserve for two <FEAT>
        )
        encoded_sequence = encoded_tweet + [feat_sep_token_id] + feature_tokens
    else:
        # Model 1: BERT*(tweet)
        encoded_sequence = tokenizer.encode(
            tweet,
            add_special_tokens=True,
            truncation=True,
            max_length=tokenizer.model_max_length
        )

    # Truncate to model's max length
    encoded_sequence = encoded_sequence[:tokenizer.model_max_length]

    # Create attention mask
    attention_mask = [1] * len(encoded_sequence)

    # Pad if necessary
    padding_length = tokenizer.model_max_length - len(encoded_sequence)
    if padding_length > 0:
        encoded_sequence += [tokenizer.pad_token_id] * padding_length
        attention_mask += [0] * padding_length

    # Assign to example
    example['input_ids'] = encoded_sequence
    example['attention_mask'] = attention_mask
    # Rename 'label' to 'labels' to match model expectations
    example['labels'] = example['label']
    del example['label']  # Remove original 'label'

    return example

def preprocess_function(example, tokenizer, model_id, model_args_dict, model_hyperparameters):
    return encode_with_tokens_based_on_model_id(example, tokenizer, model_id, model_args_dict, model_hyperparameters)

def prepare_datasets(train_dataset, val_dataset, test_dataset, tokenizer, model_id, model_args_dict, model_hyperparameters):
    # Apply the preprocessing function to all datasets
    train_dataset = train_dataset.map(
        lambda x: preprocess_function(x, tokenizer, model_id, model_args_dict, model_hyperparameters),
        batched=False
    )
    val_dataset = val_dataset.map(
        lambda x: preprocess_function(x, tokenizer, model_id, model_args_dict, model_hyperparameters),
        batched=False
    )
    test_dataset = test_dataset.map(
        lambda x: preprocess_function(x, tokenizer, model_id, model_args_dict, model_hyperparameters),
        batched=False
    )

    # Set format for PyTorch
    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

    return train_dataset, val_dataset, test_dataset

# -----------------------
# Define Metrics for Evaluation
# -----------------------

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, predictions)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

# -----------------------
# Training and Evaluation Functions
# -----------------------

def train_model(train_dataset, val_dataset, model_id, model_hyperparameters, model_args_dict, wandb_project_name):
    # Initialize wandb run
    current_time = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    run_name = f"Model_{model_id}_{current_time}"
    tags = [
        f"model_{model_id}",
        f"date_{current_time.split('_')[0]}",
    ]
    config_stuff = {**model_args_dict[model_id], **model_hyperparameters[model_id]}
    for param in config_stuff:
        tags.append(f"{param}_{config_stuff[param]}")

    run = wandb.init(
        project=wandb_project_name,
        name=run_name,
        notes=model_descriptions[model_id],
        tags=tags,
        config=config_stuff
    )

    # Initialize model and tokenizer
    special_tokens_dict = create_special_tokens_dict(model_id, model_args_dict)
    model, tokenizer = initialize_model_with_special_tokens(model_id, model_args_dict, model_hyperparameters, special_tokens_dict, model_descriptions)

    # Preprocess datasets
    train_dataset_processed, val_dataset_processed, _ = prepare_datasets(
        train_dataset, val_dataset, test_dataset, tokenizer, model_id, model_args_dict, model_hyperparameters
    )

    # Data collator (handles dynamic padding)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=model_args_dict[model_id]['output_dir'],
        overwrite_output_dir=True,
        num_train_epochs=model_hyperparameters[model_id]['num_train_epochs'],
        per_device_train_batch_size=model_hyperparameters[model_id]['train_batch_size'],
        per_device_eval_batch_size=model_hyperparameters[model_id]['train_batch_size'],
        evaluation_strategy="steps",
        save_steps=100,
        eval_steps=100,
        save_total_limit=1,
        logging_steps=50,
        learning_rate=model_hyperparameters[model_id]['learning_rate'],
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        report_to=["wandb"],
        remove_unused_columns=False  # Critical for custom datasets
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset_processed,
        eval_dataset=val_dataset_processed,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Watch the model with wandb
    wandb.watch(model, log='all')

    # Train the model
    trainer.train()

    # Evaluate the model on validation data
    eval_results = trainer.evaluate()
    wandb.log(eval_results)

    # Log an example input
    example = train_dataset_processed[0]['input_ids'].tolist()
    wandb.log({'example_input_ids': example})

    # Save artifact
    artifact = wandb.Artifact(
        name=f"model_{model_id}_{run.id}",
        type='model',
        description=model_descriptions[model_id],
        metadata={
            'model_id': model_id,
            'model_hyperparams': model_hyperparameters[model_id],
            'model_args': model_args_dict[model_id],
            'random_state': random_state,
        }
    )
    artifact.add_dir(model_args_dict[model_id]['output_dir'])
    run.log_artifact(artifact)

    # Finish wandb run
    run.finish()

    return model, tokenizer

def evaluate_model(test_dataset, model_id, model, tokenizer, model_hyperparameters, model_args_dict, wandb_project_name):
    # Preprocess test dataset
    # Note: train_dataset and val_dataset are not used here, but needed for the function signature
    # To avoid confusion, pass dummy datasets
    dummy_train, dummy_val, test_dataset_processed = prepare_datasets(
        train_dataset, val_dataset, test_dataset, tokenizer, model_id, model_args_dict, model_hyperparameters
    )

    # Data collator
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # Define evaluation arguments
    eval_args = TrainingArguments(
        output_dir=model_args_dict[model_id]['output_dir'],
        per_device_eval_batch_size=model_hyperparameters[model_id]['train_batch_size'],
        report_to=["wandb"],
        logging_steps=50
    )

    # Initialize Trainer for evaluation
    trainer = Trainer(
        model=model,
        args=eval_args,
        eval_dataset=test_dataset_processed,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Evaluate the model
    eval_results = trainer.evaluate()
    wandb.log(eval_results)

    # Get predictions
    predictions = trainer.predict(test_dataset_processed)
    preds = np.argmax(predictions.predictions, axis=1)
    true_labels = predictions.label_ids

    # Plot confusion matrix
    cm = confusion_matrix(true_labels, preds)
    plt.figure(figsize=(6,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    wandb.log({"confusion_matrix": wandb.Image(plt)})
    plt.close()

# -----------------------
# Sweep Configuration and Execution (Optional)
# -----------------------

def sweep_train():
    # Initialize the run for this set of hyperparameters
    run = wandb.init()  # This initializes the sweep run

    config = wandb.config
    chosen_model_id = config.get('model_id', 1)

    # Update hyperparameters based on config
    if 'learning_rate' in config:
        model_hyperparameters[chosen_model_id]['learning_rate'] = config.learning_rate
    if 'train_batch_size' in config:
        model_hyperparameters[chosen_model_id]['train_batch_size'] = config.train_batch_size
    if 'num_train_epochs' in config:
        model_hyperparameters[chosen_model_id]['num_train_epochs'] = config.num_train_epochs
    if 'C_ids' in config:
        model_hyperparameters[chosen_model_id]['C_ids'] = config.C_ids

    # Train the model
    model, tokenizer = train_model(
        train_dataset,
        val_dataset,
        chosen_model_id,
        model_hyperparameters,
        model_args_dict,
        WANDB_PROJECT_NAME
    )

    # Evaluate the model
    evaluate_model(
        test_dataset,
        chosen_model_id,
        model,
        tokenizer,
        model_hyperparameters,
        model_args_dict,
        WANDB_PROJECT_NAME
    )

    # Finish the run once everything is done
    wandb.finish()

sweep_config = {
    'method': 'grid',
    'metric': {'name': 'f1', 'goal': 'maximize'},  # Changed to 'f1' for better evaluation
    'parameters': {
        'model_id': {
            'values': [2]
        },
        'learning_rate': {
            'values': [1e-5, 3e-5, 5e-5]
        },
        'train_batch_size': {
            'values': [16, 32]
        },
        'C_ids': {
            'values': [[1,2], [1,2,3]]
        }
    }
}

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
model, tokenizer = train_model(
    train_dataset,   # Use the converted Dataset
    val_dataset,     # Use the converted Dataset
    2,               # model_id
    model_hyperparameters,
    model_args_dict,
    WANDB_PROJECT_NAME
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/20712 [00:00<?, ? examples/s]

Map:   0%|          | 0/2302 [00:00<?, ? examples/s]

Map:   0%|          | 0/2558 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-53-cd10f23a1b6d>:354: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,0.460100,0.447144,0.792354,0.795096,0.788194,0.791630
200,0.396800,0.383951,0.814075,0.802170,0.834201,0.817872
300,0.376900,0.381033,0.825804,0.803557,0.862847,0.832147
400,0.321900,0.317203,0.849262,0.849696,0.848958,0.849327


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,0.460100,0.447144,0.792354,0.795096,0.788194,0.791630
200,0.396800,0.383951,0.814075,0.802170,0.834201,0.817872
300,0.376900,0.381033,0.825804,0.803557,0.862847,0.832147
400,0.321900,0.317203,0.849262,0.849696,0.848958,0.849327
500,0.311400,0.314858,0.847958,0.854240,0.839410,0.846760
600,0.281800,0.295152,0.858818,0.841453,0.884549,0.862463
700,0.296400,0.283255,0.870982,0.840637,0.915799,0.876610
800,0.293100,0.294451,0.867072,0.880396,0.849826,0.864841
900,0.268300,0.292138,0.861859,0.904070,0.809896,0.854396
1000,0.257900,0.282686,0.869679,0.840800,0.912326,0.875104


wandb: Adding directory to artifact (./type2)... Done. 9.5s


epoch,▁
eval/accuracy,▁▃▄▆▆▇██▇██
eval/f1,▁▃▄▆▆▇█▇▆██
eval/loss,█▅▅▂▂▂▁▂▁▁▁
eval/precision,▁▁▂▅▅▄▄▆█▄▄
eval/recall,▁▄▅▄▄▆█▄▂██
eval/runtime,▃▁█▇██████▆
eval/samples_per_second,▆█▁▂▁▁▁▁▁▁▃
eval/steps_per_second,▆█▁▂▁▁▁▁▁▁▃
eval_accuracy,▁
eval_f1,▁


In [ ]:
model, tokenizer = train_model(
    train_dataset,   # Use the converted Dataset
    val_dataset,     # Use the converted Dataset
    3,               # model_id
    model_hyperparameters,
    model_args_dict,
    WANDB_PROJECT_NAME
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Map:   0%|          | 0/20712 [00:00<?, ? examples/s]

Map:   0%|          | 0/2302 [00:00<?, ? examples/s]

Map:   0%|          | 0/2558 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-53-cd10f23a1b6d>:354: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,0.462600,0.463464,0.768897,0.797505,0.721354,0.757521
200,0.411900,0.485517,0.738923,0.848293,0.582465,0.690685
300,0.385600,0.429857,0.813640,0.772419,0.889757,0.826946
400,0.367000,0.336861,0.846655,0.802881,0.919271,0.857143
500,0.333800,0.331746,0.843614,0.847368,0.838542,0.842932
600,0.312600,0.303200,0.851868,0.819040,0.903646,0.859265
700,0.293800,0.292429,0.870547,0.836220,0.921875,0.876961
800,0.323600,0.318618,0.855778,0.894231,0.807292,0.848540
900,0.283000,0.285469,0.870547,0.870017,0.871528,0.870772
1000,0.288300,0.292949,0.868375,0.842615,0.906250,0.873275


wandb: Adding directory to artifact (./type3)... Done. 42.9s


epoch,▁
eval/accuracy,▃▁▅▇▇▇█▇███
eval/f1,▄▁▆▇▇▇█▇███
eval/loss,▇█▆▃▃▂▁▂▁▁▁
eval/precision,▂▅▁▃▅▄▅█▇▅▅
eval/recall,▄▁▇█▆██▆▇██
eval/runtime,▁▃▅▅▅▅▅▅▅▆█
eval/samples_per_second,█▆▄▄▄▄▄▃▄▃▁
eval/steps_per_second,█▆▄▄▄▄▄▄▄▄▁
eval_accuracy,▁
eval_f1,▁
